In [1]:
# example. using local meta store, using local  to store data
# meta store is not usable in other notebooks, as it is embedeed locked jvm internally


In [ ]:
import findspark
findspark.init()

In [ ]:
"""
Spark Database 
Basic Database:
Single , local database, not shared across multiple notebooks,
not shared across worker, in  built into spark working director,
generally used for simple development/learning purpose

In production, you will be using Hive Data Catalog
"""

In [ ]:
"""
Spark Local Database
Only for dev only, not for production

3 components involved

1. meta data - database name, tables, columns data types, location where data stored
    is managed by hive, hive internally uses derby db to store all meta data
2. spark temporary location  "spark.local.dir", "/home/ubuntu/spark-temp"
    where temp data used internally stored
    
3. "spark.sql.warehouse.dir", "/home/ubuntu/spark-warehouse" spark data warehouse where all the database data shall be stored
    we can see database, tables, their data where meta data ,table name, columns are stored in 
    meta data
    
"""

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkDatabase")

# embedded, simple, local spark database/warehouse
# spark will store temporary files
# enable hive support must for sql database
# enable hiveSupport hive catalog to be embedded inside working directory
# spark temp data goes to "/home/ubuntu/spark-temp"
config.set("spark.local.dir", "/home/ubuntu/spark-temp")
# spark data [not meta data] goes into  "/home/ubuntu/spark-warehouse"
config.set("spark.sql.warehouse.dir", "/home/ubuntu/spark-warehouse")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame

# enableHiveSupport() create a meta catalog/database using derby database
# inside current working directory, embedded into spark notebook,
# multiple notebooks cannot share at same time.
# inside pyspark-notebooks, you could see metastore_db
# metastore shall have meta data: database, tables, columns, data types, where exactly
# data located in hdfs or file system or s3
# derby.log - derby database log 
## metastore_db 

spark = SparkSession.builder\
                    .config(conf=config)\
                    .enableHiveSupport()\
                    .getOrCreate()

sc = spark.sparkContext

In [3]:
df = spark.sql("SHOW DATABASES")

df.show()

AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

In [11]:
spark.sql("CREATE DATABASE IF NOT EXISTS stocklocaldb")

DataFrame[]

In [12]:
# Create spark Managed table
# we have to use spark sql like insert, (update, delete won't work at 2.x)
# to add data
spark.sql("CREATE TABLE  IF NOT EXISTS stocks(symbol STRING, industry STRING)")

DataFrame[]

In [7]:
spark.sql("""
 INSERT INTO stocks VALUES('INFY', 'IT')
"""
         )

DataFrame[]

In [8]:
spark.sql("SELECT * FROM stocks").show()

+------+--------+
|symbol|industry|
+------+--------+
|  INFY|      IT|
+------+--------+



In [13]:
# this command drop the table from meta data store and drop the in the 
# spark datawarehouse directory

spark.sql("DROP TABLE IF EXISTS stocks")

DataFrame[]

In [14]:
spark.sql("SHOW TABLES").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [15]:
# if no table exists, no data inside, then it drop the database
# drop the metadata too..
spark.sql("DROP DATABASE IF EXISTS stocklocaldb")

DataFrame[]

In [16]:
spark.sql("SHOW DATABASES").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [18]:
spark.sql("SHOW DATABASES").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [19]:
# create databsae called productdb
# /home/ubuntu/spark-warehouse/productdb.db
spark.sql("""CREATE DATABASE IF NOT EXISTS productdb""")

DataFrame[]

In [20]:
# create a table called products with 3 fields (id int, name string, price int)
# create table productdb.products ...
# create in ubuntu /home/ubuntu/spark-warehouse/productdb.db/products
spark.sql("""
CREATE TABLE IF NOT EXISTS 
    productdb.products( id INT, 
                        product_name STRING, 
                        price INT )
"""
)

DataFrame[]

In [21]:
# insert at least 2 records productdb.products
# created in ubuntu /home/ubuntu/spark-warehouse/productdb.db/products/part*

spark.sql("""
INSERT INTO productdb.products VALUES(10, 'Cheese', 3), 
                                        (25, 'Apricot', 2)
"""
         )

DataFrame[]

In [23]:
# query data using productdb.produces 
spark.sql("SELECT * FROM productdb.products").show()

+---+------------+-----+
| id|product_name|price|
+---+------------+-----+
| 10|      Cheese|    3|
| 25|     Apricot|    2|
+---+------------+-----+



In [22]:
# use where condition in query too

spark.sql("SELECT * FROM productdb.products WHERE product_name LIKE 'A%'").show()


+---+------------+-----+
| id|product_name|price|
+---+------------+-----+
| 25|     Apricot|    2|
+---+------------+-----+

